# Accesing Watsonx.ai via REST API

In this lab, we will take a comprehensive look at making HTTP requests to access [Watsonx.ai's REST API](https://workbench.res.ibm.com/docs/api-reference) and learn how to use the functionality.  This lab explore only a few of the many REST endpoints available so explore the REST API documentation to view the full list of capabilities.

In [5]:
from dotenv import load_dotenv
import json
import os
import requests

## HTTP request headers
Headers contain parameter values that represent the metadata associated with an API requests and response. In the following example, the Authorization header provides the server with credentials to validate your access.  Watsonx.ai uses a "Bearer" access token wich is used to pass our Watsonx.ai authentication key.  The 'Content-type' header in the request is added to tell the server or the browser which is serving the resource to the end user about the media type of the request. In this case, type of expected data as 'application/json'.

In [8]:
load_dotenv()
api_key = os.getenv("GENAI_KEY", None)
api_endpoint = os.getenv("GENAI_API", None)
if api_key is None or api_endpoint is None:
    print("Ensure you copied the .env file that you created earlier into the same directory as this notebook")
else:
    headers={"Authorization": f"Bearer {api_key}",
             "Content-Type":"application/json"}

## POST vs GET
HTTP requests come in two flavors: GET and POST.  When using GET, data parameters are included in the URL and visible to everyone. However, when using POST, data is not displayed in the URL but is instead passed in the HTTP message body. 

GET requests are intended to retrieve data from a server and do not modify the server’s state. On the other hand, POST requests are used to send data to the server for processing and may modify the server’s state.  

## POST requests with 'Generate' endpoint

The generate endpoint "https://workbench-api.res.ibm.com/v1/generate" provides an interface for sending prompts to any model supported by Watsonx.ai. Given a text prompt as inputs, and required parameters, the selected model will attempt to complete the provide input and return "generated_text".

Request body needs to include:
- Model id (string): the id of the model
- Inputs (array of strings): prompts to generate completions
- Parameters for the model (key-value pairs)


In [9]:
body={
  "model_id": "google/flan-ul2",
  "inputs": ["Write a short blog post for an advanced cloud service for large language models: This service is"],
  "parameters": {
    "temperature": 0,
    "max_new_tokens": 50,
    "min_new_tokens": 25
  }
}

In [11]:
url=f"{api_endpoint}generate" 
response=requests.post(url=url, headers=headers, json=body )
print("JSON Response: ",response.json())


JSON Response:  {'model_id': 'google/flan-ul2', 'created_at': '2023-06-30T14:42:20.761Z', 'results': [{'generated_text': 'a new cloud service for large language models. It is a scalable, high-performance, and secure service for training and deploying language models.', 'generated_token_count': 33, 'input_token_count': 20, 'stop_reason': 'EOS_TOKEN'}]}


In [12]:
print(f"Model results: {json.dumps(response.json()['results'], indent = 2)}") 

Model results: [
  {
    "generated_text": "a new cloud service for large language models. It is a scalable, high-performance, and secure service for training and deploying language models.",
    "generated_token_count": 33,
    "input_token_count": 20,
    "stop_reason": "EOS_TOKEN"
  }
]


## Using GET requests to retrieve data
The GET method is used to retrieve information from Watsonx.ai using a given URL.

### GET /models
Get the [list of all models](https://workbench.res.ibm.com/docs/api-reference#list-models) supported by Watsonx.ai

In [46]:
WATSONX_AI_BASE_URL = "https://workbench-api.res.ibm.com/v1"

url = WATSONX_AI_BASE_URL + "/models"
response = requests.get(url = url,headers=headers)
models = response.json()["results"]

print(f"{len(models)} models supported the Watsonx.ai")

# Print the first two models
models[0:2]

9 models supported the Watsonx.ai


[{'id': 'salesforce/codegen-16b-mono',
  'name': 'codegen-16b-mono (16B)',
  'size': '16B',
  'source_model_id': None,
  'token_limit': 2048},
 {'id': 'prakharz/dial-flant5-xl',
  'name': 'dial-flant5-xl (3B)',
  'size': '3B',
  'source_model_id': None,
  'token_limit': 4096}]

### GET /models/{id}
Retrieve [details on a specific model](https://workbench.res.ibm.com/docs/api-reference#list-models) supported by Watsonx.ai

In [48]:
model_id = models[0]["id"]
print(f"initial model id: {model_id}")

# Some model IDs contain forward slashes which can't be directly appended to the url.
# These slashes must first be "escaped" to % encoding using the ASCII character-set.
# More escape encoding can be found here: https://www.w3schools.com/tags/ref_urlencode.ASP

# change '/' in model id to '%2F'
model_id = model_id.replace("/","%2F")
print(f"escaped model id: {model_id}")

url = WATSONX_AI_BASE_URL + "/models/" + model_id
response = requests.get(url=url, headers=headers)
model_info = response.json()
model_info

initial model id: salesforce/codegen-16b-mono
escaped model id: salesforce%2Fcodegen-16b-mono


{'results': {'id': 'salesforce/codegen-16b-mono',
  'name': 'codegen-16b-mono (16B)',
  'size': '16B',
  'disabled': False,
  'preferred': True,
  'description': 'codegen-16b-mono (16B) is an autoregressive language model for program synthesis trained sequentially on the Pile, BigQuery and BigPython. codegen-mono (16B) is trained with a 16 billion parameter checkpoint.\n\n- Repository: [salesforce/CodeGen](https://github.com/salesforce/CodeGen)\n- Paper: [A Conversational Paradigm for Program Synthesis](https://arxiv.org/abs/2203.13474)\n- More Information: [from Huggingface](https://huggingface.co/docs/transformers/model_doc/codegen)\n- License: [BSD 3-Clause](https://github.com/salesforce/CodeGen/blob/main/LICENSE.txt)\n- Intended Use: \n    - Extracting features from given natural language and programming language texts, and calculating the likelihood. \n    - Programming synthesis-generating executable code given English prompts, where the prompts should be in the form of a comment

### Lab Complete
Don't forget to explore the [Watsonx.ai's REST API](https://workbench.res.ibm.com/docs/api-reference) to explore the the full list of capabilities.